### setup
1. use the dataset available at open datasets at microsoft https://learn.microsoft.com/de-de/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark 
2. save the first 2000 rows as table to unity catalog
3. calcuate the trip duration 
4. save the cleaned data as a new table to be used for auto ml training

In [0]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = "r"

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

In [0]:
df.limit(2000).write.format("delta").mode("overwrite").saveAsTable("main_sj.linkedinlearning.taxicomplete")

In [0]:
# Calculate trip duration
df_trip_duration = spark.sql("""
SELECT *,
       (unix_timestamp(tpepDropoffDateTime) - unix_timestamp(tpepPickupDateTime)) / 60 AS trip_duration_minutes
FROM main_sj.linkedinlearning.taxicomplete
""")

# Display the result
display(df_trip_duration)

In [0]:
df_trip_duration.write.format("delta").mode("overwrite").saveAsTable("main_sj.linkedinlearning.taxicomplete_trip_duration")

In [0]:
df_cleaned = spark.sql("""
SELECT vendorID, passengerCount, tripDistance, paymentType, fareAmount, extra, mtaTax, tipAmount, tollsAmount, trip_duration_minutes
FROM main_sj.linkedinlearning.taxicomplete_trip_duration
""")

df_cleaned.write.format("delta").mode("overwrite").saveAsTable("main_sj.linkedinlearning.cleaneddata")